# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.48.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'MPG-Prediction'

experiment=Experiment(ws, experiment_name)


quick-starts-ws-230710
aml-quickstarts-230710
westus2
f9d5a085-54dc-4215-9ba6-dad5d86e60a0


### Create or Attach an AmlCompute cluster

In [6]:

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


#### Data

details:...

In [13]:
dataset = Dataset.get_by_name(ws, name='mpg_pred')
#dataset.to_pandas_dataframe()

#view the data
dataset.take(5).to_pandas_dataframe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year
0,18.0,8,307.0,130,3504,12.0,70
1,15.0,8,350.0,165,3693,11.5,70
2,18.0,8,318.0,150,3436,11.0,70
3,16.0,8,304.0,150,3433,12.0,70
4,17.0,8,302.0,140,3449,10.5,70


In [14]:

# Split the dataset into train and test datasets
train_data, test_data = dataset.random_split(percentage=0.8, seed=10)

label = "mpg"

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:
# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": "r2_score",
    "enable_early_stopping": True,
    "max_concurrent_iterations": 5,
    "experiment_timeout_minutes": 30
   
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="regression",
    compute_target=compute_target,
    training_data=train_data,
    label_column_name=label,
    **automl_settings,
)


In [16]:
# TODO: Submit your experiment

remote_run = experiment.submit(automl_config, show_output= True)


Submitting remote run.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml


Experiment,Id,Type,Status,Details Page,Docs Page
MPG-Prediction,AutoML_a303ff96-088a-4ede-99d1-6bbafcfa329b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [18]:
remote_run.wait_for_completion()

{'runId': 'AutoML_a303ff96-088a-4ede-99d1-6bbafcfa329b',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2023-04-11T05:03:11.446662Z',
 'endTimeUtc': '2023-04-11T05:17:48.84534Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"MPG-Prediction","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-230710","workspace_name":"quick-starts-ws-230710","region":"westus2","compute_target":"auto-ml","spark_service":null,"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [20]:
#TODO: Save the best model

best_run, fitted_model = remote_run.get_output()

print("best run details :",best_run.get_details())
print("best run file names :",best_run.get_file_names())
print("best run metrics :",best_run.get_metrics())

#save the model
best_run.register_model(model_name = "AutoML_best_run", model_path = 'outputs/')

print(best_run)

best run details : {'runId': 'AutoML_a303ff96-088a-4ede-99d1-6bbafcfa329b_38', 'target': 'auto-ml', 'status': 'Completed', 'startTimeUtc': '2023-04-11T05:16:45.540728Z', 'endTimeUtc': '2023-04-11T05:17:42.776048Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'regression\',\'primary_metric\':\'r2_score\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'MPG-Prediction\',\'compute_target\':\'auto-ml\',\'subscription_id\':\'f9d5a085-54dc-4215-9ba6-dad5d86e60a0\',\'region\':\'westus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_a303ff96-088a-4ede-99d1-6bbafcfa329b_38","experiment_name":"MPG-Prediction","workspace_name":"quick-starts-ws-230710","subscription_id":"f9d5a085-54dc-4215

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
